# **Dataset Name:** Shark Attack
# **Dataset Source:** https://mavenanalytics.io/data-playground?search=shark
# **Dataset Description**
# This is a dataset of Shark attacks reported over the past 100 years, including location (country & city), activity, injury type, victim info (name, gender, age), shark species, etc.The dataset consists of 22 fields & 25614 records. There are both Numerical and Categorical data in this dataset. Most of the data are Categorical. The remaining Numerical Data are also in object form which need preprocessing to convert them into integer. This dataset was posted on 2020.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import plotly.express as px
import plotly.graph_objects as go

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score, f1_score
from collections import Counter
from datetime import datetime

# **Load the Dataset**

In [ ]:
atk = pd.read_csv("/content/drive/MyDrive/Datasets/shark_attacks.csv", encoding="latin-1")
backup_atk = copy.deepcopy(atk)


# **Overview of Dataset**

In [ ]:
atk.head()
atk.shape
atk.info()
atk.describe()

# **Deleting unnecessary columns**

In [ ]:
# deleting columns
column_to_delete = {'Case Number', 'Investigator or Source', 'pdf', 'href', 'href formula','Case Number', 'original order', 'Area', 'Case Number.1', 'Case Number.2'}
for x in column_to_delete:
  if x in atk.columns:
      atk.drop(x, axis=1, inplace=True)
      print(f"Column '{x}' deleted successfully.")
  else:
      print(f"Column '{x}' not found.")


# Save the modified DataFrame to a new CSV file
atk.to_csv('sharkAttacks.csv', index=False)

# **Merging identical columns**

In [ ]:
atk = atk.groupby(level=0, axis=1).first()

# Save the merged DataFrame to a new CSV file
atk.to_csv('shark_attacks.csv', index=False)

# **Label Encoding**

In [ ]:
# Label encoding to get numeric value from Sex and Fatal column
sex_label_encoder = LabelEncoder()
atk['sex_numeric'] = sex_label_encoder.fit_transform(atk['Sex'])

fatal_label_encoder = LabelEncoder()
atk['fatal_numeric'] = fatal_label_encoder.fit_transform(atk['Fatal'])

# **Data Preprocessing**

In [ ]:
# Renaming faulty column names

atk.rename(columns={'Sex ': 'Sex'}, inplace=True)
atk.rename(columns={'Species ': 'Species'}, inplace=True)
atk.rename(columns={'Fatal (Y/N)': 'Fatal'}, inplace=True)

atk.columns

In [ ]:
# Converting Age data type from object to int

# drop rows with non-integer values
atk = atk.dropna(subset=['Age'])

# convert non-integer values to NaN
atk['Age'] = pd.to_numeric(atk['Age'], errors='coerce')

# impute missing values
atk['Age'].fillna(atk['Age'].mean(), inplace=True)

# conversion to int
atk['Age'] = atk['Age'].astype(int)

In [ ]:
# converting year data type from float to int

# drop rows with non-integer values
atk = atk.dropna(subset=['Year'])

# convert non-integer values to NaN
atk['Year'] = pd.to_numeric(atk['Year'], errors='coerce')

# impute missing values
atk['Year'].fillna(atk['Year'].mean(), inplace=True)

# conversion to int
atk['Year'] = atk['Year'].astype(int)
atk.dtypes

In [ ]:
# preprocessing DATE
def date(date):
  months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
  order = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

  date = str(date)
  if (len(date) == 11 and date[2] == '-'):
    for i in range(len(months)):
      date = date.replace(months[i], order[i])    # replacing month name with its order number
    return date
  else:
    return np.nan

In [ ]:
# preprocessing YEAR
def year(year):
    if year > 1800:
      return year
    else:
      return np.nan


In [ ]:
# preprocessing SEX
def sex(sex):
  if sex == 'M' or sex == 'F':
    return sex
  else:
    return np.nan

In [ ]:
# preprocessing AGE
def age(age):
  try:
    age = int(age)
  except ValueError:
    age = 0

  if age > 0 and age <= 100:
    return age
  else:
    return np.nan

In [ ]:
# preprocessing FATAL
def fatal(fatal):
  if fatal == 'N' or fatal == 'Y':
    return fatal
  else:
    return np.nan

In [ ]:
#calling the preprocessing functions

#atk['Date'] = atk['Date'].apply(date)
atk['Year'] = atk['Year'].apply(year)
atk['Sex'] = atk['Sex'].apply(sex)
atk['Age'].fillna(0, inplace=True)
atk['Age'] = atk['Age'].apply(age)
atk['Fatal'] = atk['Fatal'].apply(fatal)
#atk.describe()

# **Analysis based on Age**

In [ ]:
plt.hist(atk['Age'], bins=10, color='skyblue', edgecolor='black')

# Add labels and title
plt.xlabel('Age')
plt.ylabel('Count')
plt.title('Histogram')

# Show plot
plt.show()

# **Analysis Based on Fataliity**


In [ ]:
fatal_vals = atk['Fatal'].value_counts().tolist()

f, ax = plt.subplots(figsize=(5, 5))

labels = ['Not Fatal', 'Fatal']
colors = ['green', 'red']

plt.pie(fatal_vals, labels=labels, colors=colors,
        autopct='%1.1f%%', startangle=90)

axis = plt.axis('equal')

# **Analysis based on Shark Species**

In [ ]:
from_re = [r'.*?\bwhite\b\s+\bshark\b.*',r'.*?\bblue\b\s+\bshark\b.*', r'.*?\btiger\b\s.*',
           r'.*?\bbull\b\s.*',r'.*?\bshark\b\s+\binvolvement\b.*',r'.*?\bwobbegong\b\s+\bshark\b.*',
           r'.*?\bblacktip\b\s.*', r'.*?\bbronze\b\s+\bwhaler\b.*', r'.*?\bmako\b\s.*',r'.*?\bnurse\b\s.*',
           r'.*?\bhammerhead\b\s.*', r'.*?\braggedtooth\b\s.*']


to_re = ['White shark', 'Blue shark', 'Tiger shark',
         'Bull shark', 'Not a shark', 'Wobbegong shark',
         'Blacktip shark','Bronze whaler shark', 'Mako shark',
         'Nurse shark', 'Hammerhead shark', 'Raggedtooth shark']

atk.Species = atk.Species.str.lower().replace(from_re, to_re, regex=True)

top = atk.Species.value_counts().head(7)

sns.set(rc={'axes.facecolor':'w', 'figure.facecolor':'white'})
top.plot(kind='bar', figsize=(6, 4), alpha=0.9, color='skyblue', rot=45, fontsize=9)
plt.xlabel('Species')
plt.ylabel('Number of cases', fontsize=10)
plt.title('Top 7 most dangerous species of shark')
plt.grid(color='k', axis='y', alpha=0.4)


plt.tight_layout()
plt.show()

# **Analysis based on Activity**

In [ ]:
prov_activity = atk[atk.Type == 'Provoked'].groupby('Activity')['Activity'].count().sort_values(ascending=False)[:10]

fig = px.bar(prov_activity, x=prov_activity.values, y=prov_activity.index, orientation='h', labels={'index':'','x':'Attack Count'},
            title = 'Provoked Attacks by Activity')
fig.update_layout(height=500, width=800)
fig.show()

atk['Species'].value_counts()
prov_activity.describe()
prov_activity.head()

# **Analysis based on Sex**

In [ ]:
sex_vals = atk['Sex'].value_counts().tolist()

f, ax = plt.subplots(figsize=(6, 6))

labels = ['Male', 'Female', 'Unknown', 'Not reported', 'Invalid', 'No data']
colors = ['blue', 'pink', 'grey', 'orange', 'red', 'black']

plt.pie(sex_vals, labels=labels, colors=colors,
        autopct='%1.1f%%', startangle=90)

axis = plt.axis('equal')

# **Analysis based on Years**

In [ ]:
# Convert 'Date' column to datetime with flexible format
atk['Date'] = pd.to_datetime(atk['Date'], infer_datetime_format=True, errors='coerce')

# Drop rows with missing or incorrect dates
atk = atk.dropna(subset=['Date'])

# Extract year from 'Date' column
atk['Year'] = atk['Date'].dt.year

# Group data by 'Year' and count incidents
yearly_incidents = atk.groupby('Year').size()

plt.figure(figsize=(12, 6))
plt.plot(yearly_incidents.index, yearly_incidents.values, marker='o')
plt.title('Shark Attack Incidents Over the Years')
plt.xlabel('Year')
plt.ylabel('Number of Incidents')
plt.grid(True)
plt.show()

# **Analysis based on Injury type**


In [ ]:
plt.figure(figsize=(8, 6))
sns.countplot(x='Injury', data=atk)
plt.title('Distribution of Injury Severity')
plt.xlabel('Injury Severity')
plt.ylabel('Count')
plt.show()

# **Analysis based on Country**

In [ ]:
countryCount = atk['Country'].value_counts().reset_index()
countryCount.head()

In [ ]:
fig = px.choropleth(data_frame = countryCount,
                    locations = 'Country',
                    color = 'count',
                    locationmode = 'country names',
                    scope = 'world',
                    title = 'Shark Attack around the World')

fig.show()

In [ ]:
plt.figure(dpi=120)
sns.barplot(data=countryCount.head(5), y='count', x='Country', palette='Set2')
plt.title('Top 5 Countries with Highest number of Shark Attack cases')
plt.show()

# **Bivariate Analysis on Sex and Fatality**

In [ ]:
biv = atk.groupby(['Sex', 'Fatal'],as_index=False).size()
biv = biv.sort_values(by=['size'], ascending=False)
biv.head()

In [ ]:
mlabels=['Male Non Fatal', 'Male Fatal','Female Non Fatal','Female Fatal']
plt.pie(biv['size'],labels=mlabels,autopct='%1.1f%%')
plt.title('comparison of fatal/non fatal accidents among women and men')
fig.set_size_inches(12,12)
plt.show()

# **Bivariate Analysis based on Species and Fatality**

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x=atk['Species'], y=atk['Fatal'], data=data, color='blue')
plt.title('Scatter plot of variable1 vs variable2')
plt.xlabel(str(atk['Species']))
plt.ylabel(str(atk['Fatal']))
plt.show()

# **Bivariate Analysis based on Country and Fatality**

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x=atk['Country'], y=atk['Fatal'], data=data, color='blue')
plt.title('Scatter plot of variable1 vs variable2')
plt.xlabel(str(atk['Country']))
plt.ylabel(str(atk['Fatal']))
plt.show()

# **Multivariate Analysis using PairPlot**

In [ ]:
sns.pairplot(atk)
plt.show()

# **One Hot Encoding**

In [ ]:
#print(atk.dtypes)

# Perform one hot encoding on key features
columns_to_encode1 = ['Country', 'Species']
ohe1 = pd.get_dummies(atk, columns = columns_to_encode1)

columns_to_encode2 = ['Country','Sex']
ohe2 = pd.get_dummies(atk, columns = columns_to_encode2)

columns_to_encode3 = ['Country', 'Type']
ohe3 = pd.get_dummies(atk, columns = columns_to_encode3)

# Display the encoded DataFrame
print("Encoded DataFrame:")
print(ohe1)

In [ ]:
print(ohe2)

In [ ]:
print(ohe3)

# **Correlation and Heatmap**


In [ ]:
sns.heatmap(atk.corr(method='pearson', min_periods=1, numeric_only=True), annot = True)
sns.pairplot(atk)

# **Model Training**

# Classification


In [ ]:
# handle NaN values and drop them
atk.isnull().sum()
atk = atk.dropna()

# impute missing values
imputer = SimpleImputer(strategy="mean")
atk[["Year", "Age", "sex_numeric"]] = imputer.fit_transform(atk[["Year", "Age", "sex_numeric"]])

X = atk[['Year', 'Age', 'sex_numeric']]  # Features
y = atk['Fatal']  # Target variable

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the logistic regression model
model = LogisticRegression()

# Train the model on the training data
model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Display classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Neural Network

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow import keras
from tensorflow.keras import layers
from google.colab import drive

data = pd.read_csv('/content/drive/MyDrive/Datasets/shark_attacks.csv', usecols=['Year', 'Country', 'Sex ', 'Age', 'Fatal (Y/N)'], encoding='latin1')
data.dropna(inplace=True)

data = data[data['Country'].notna()]
data = data[data['Sex '].notna()]
data = data[data['Sex '].isin(['M', 'F'])]
data = data[data['Age'].notna()]
data = data[~data['Age'].str.contains(r'[a-zA-Z]|&|\?')]

data['Fatal (Y/N)'] = data['Fatal (Y/N)'].apply(lambda x: 1 if x == 'Y' else 0)

label_encoder = LabelEncoder()
data['Country'] = label_encoder.fit_transform(data['Country'])
data['Sex '] = label_encoder.fit_transform(data['Sex '])

data['Age'] = data['Age'].str.replace(r'\D', '')
data['Age'] = pd.to_numeric(data['Age'], errors='coerce')

data.dropna(subset=['Age'], inplace=True)

X = data.drop('Fatal (Y/N)', axis=1)
y = data['Fatal (Y/N)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[X_train_scaled.shape[1]]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

_, accuracy = model.evaluate(X_test_scaled, y_test)
print('Accuracy:', accuracy)

y_pred_prob = model.predict(X_test_scaled)

y_pred = (y_pred_prob > 0.5).astype(int)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)